### Домашнее задание 03

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других

In [34]:
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [35]:
import numpy as np
import pandas as pd

#from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
#from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, log_loss
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

df = pd.read_csv('train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [36]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

In [37]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [38]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

In [39]:
classifierLogReg = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])

classifierRandForest = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

classifierCatBoost = Pipeline([
    ('features',feats),
    ('classifier', CatBoostClassifier(random_state = 42, silent=True)),
])

In [40]:
def cv_and_fit(classifier):
    # запустим кросс-валидацию
    cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring="roc_auc")
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)
    print(f'CV score is {cv_score}+-{cv_score_std}')

    # обучим пайплайн на всем тренировочном датасете
    classifier.fit(X_train, y_train)
    y_score = classifier.predict_proba(X_test)[:, 1]
    return y_score


def metrics(classifier):
    b = 1
    y_score = cv_and_fit(classifier)
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1 + b ** 2) * (precision * recall) / (b ** 2 * precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    roc_auc_tmp = roc_auc_score(y_test, y_score)
    log_loss_tmp = log_loss(y_test, y_score)
    # print(
    #     "Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f, Roc_auc=%.3f, Log_loss=%.3f"
    #     % (
    #         thresholds[ix],
    #         fscore[ix],
    #         precision[ix],
    #         recall[ix],
    #         roc_auc_tmp,
    #         log_loss_tmp,
    #     )
    # )
    return (
        thresholds[ix],
        fscore[ix],
        precision[ix],
        recall[ix],
        roc_auc_tmp,
        log_loss_tmp,
    )


In [41]:
all_metrics = (
    metrics(classifierLogReg),
    metrics(classifierRandForest),
    metrics(classifierCatBoost),
)

metrics_table = pd.DataFrame(
    list(all_metrics),
    columns=["tresholds", "f-score", "precision", "recall","roc-auc", "log_loss",],
    index=["LogisticRegression", "RandomForest", "CatBoost"],
)


CV score is 0.7867401104915408+-0.00852135511666111
CV score is 0.7734501681056019+-0.007171140345435727


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

CV score is 0.802753018270199+-0.0070677038787192075


In [42]:
metrics_table

,tresholds,f-score,precision,recall,roc-auc,log_loss
LogisticRegression,0.386937,0.730323,0.647431,0.837558,0.784035,0.577960
RandomForest,0.350000,0.718863,0.642669,0.815553,0.771037,0.599298
CatBoost,0.367163,0.739551,0.677302,0.814401,0.801211,0.541438


### По результатам можно сказать, что нет кардинальных различий между моделями при данных входных значениях и настройках моделей по умолчанию. Возможно более глубокий тюнинг моделей, а так же обработка входных данных могут дать разницу.